# OpenVINO Training Extension


## Import everything

In [ ]:
import os
import sys

import cv2
import numpy as np

from ote_sdk.configuration.helper import create as create_parameters_from_parameters_schema
from ote_sdk.entities.inference_parameters import InferenceParameters
from ote_sdk.entities.label_schema import LabelSchemaEntity
from ote_sdk.entities.model import ModelEntity, ModelOptimizationType
from ote_sdk.entities.resultset import ResultSetEntity
from ote_sdk.entities.subset import Subset
from ote_sdk.entities.task_environment import TaskEnvironment
from ote_sdk.entities.optimization_parameters import OptimizationParameters
from ote_sdk.usecases.adapters.model_adapter import ModelAdapter
from ote_sdk.usecases.tasks.interfaces.export_interface import ExportType
from ote_sdk.usecases.tasks.interfaces.optimization_interface import OptimizationType

from ote_cli.datasets import get_dataset_class
from ote_cli.registry import Registry
from ote_cli.utils.importing import get_impl_class
from ote_cli.utils.io import read_binary, save_model_data

## Register templates

In [ ]:
templates_dir = '../external'
registry = Registry(templates_dir)
print(registry)

## Load model template and its hyper parameters

In [ ]:
model_template = registry.get('Custom_Object_Detection_Gen3_ATSS')
hyper_parameters = model_template.hyper_parameters.data

## Get dataset instantiated

In [ ]:
Dataset = get_dataset_class(model_template.task_type)

dataset = Dataset(
    train_subset={'ann_file': '../data/airport/annotation_faces_train.json',
                  'data_root': '../data/airport/'},
    val_subset={'ann_file': '../data/airport/annotation_faces_train.json',
                'data_root': '../data/airport'}
)
labels_schema = LabelSchemaEntity.from_labels(dataset.get_labels())

## Have a look at existing parameters

In [ ]:
hyper_parameters = create_parameters_from_parameters_schema(hyper_parameters)

for p in hyper_parameters.learning_parameters.parameters:
    print(f'{p}: {getattr(hyper_parameters.learning_parameters, p)}')

## Tweak parameters

In [ ]:
hyper_parameters.learning_parameters.batch_size = 8
hyper_parameters.learning_parameters.num_iters = 15

for p in hyper_parameters.learning_parameters.parameters:
    print(f'{p}: {getattr(hyper_parameters.learning_parameters, p)}')

## Create Task 

In [ ]:
Task = get_impl_class(model_template.entrypoints.base)

environment = TaskEnvironment(
    model=None,
    hyper_parameters=hyper_parameters,
    label_schema=labels_schema,
    model_template=model_template)
        
task = Task(task_environment=environment)

## Run training

In [ ]:
torch_model = ModelEntity(
    dataset,
    environment.get_model_configuration(),
)

task.train(dataset, torch_model)

## Evaluate quality metric

In [ ]:
validation_dataset = dataset.get_subset(Subset.VALIDATION)
predicted_validation_dataset = task.infer(
    validation_dataset.with_empty_annotations(),
    InferenceParameters(is_evaluation=True))

resultset = ResultSetEntity(
    model=torch_model,
    ground_truth_dataset=validation_dataset,
    prediction_dataset=predicted_validation_dataset,
)
task.evaluate(resultset)
print(resultset.performance)

In [ ]:
save_model_data(torch_model, "./torch_model")

## OpenVINO

### Export PyTorch model to OpenVINO format

In [ ]:
exported_model = ModelEntity(
    dataset,
    environment.get_model_configuration(),
)
task.export(ExportType.OPENVINO, exported_model)
save_model_data(exported_model, "./exported_model")

### Evaluate the exported model

In [ ]:
environment.model = exported_model
ov_task = get_impl_class(model_template.entrypoints.openvino)(environment)
predicted_validation_dataset = ov_task.infer(
    validation_dataset.with_empty_annotations(),
    InferenceParameters(is_evaluation=True))
resultset = ResultSetEntity(
    model=exported_model,
    ground_truth_dataset=validation_dataset,
    prediction_dataset=predicted_validation_dataset,
)
ov_task.evaluate(resultset)
print(resultset.performance)

In [ ]:
!benchmark_app -m ./exported_model/openvino.xml  -i ../data/airport/val

## Optimize OpenVINO with POT

In [ ]:
environment.model = exported_model
pot_model = ModelEntity(
    dataset,
    environment.get_model_configuration(),
)

ov_task.optimize(
    OptimizationType.POT,
    dataset,
    pot_model,
    OptimizationParameters(),
)

save_model_data(pot_model, "./pot_model")

### Evaluate OpenVINO POT model

In [ ]:
environment.model = pot_model
ov_task = get_impl_class(model_template.entrypoints.openvino)(environment)
predicted_validation_dataset = ov_task.infer(
    validation_dataset.with_empty_annotations(),
    InferenceParameters(is_evaluation=True))
resultset = ResultSetEntity(
    model=pot_model,
    ground_truth_dataset=validation_dataset,
    prediction_dataset=predicted_validation_dataset,
)
ov_task.evaluate(resultset)
print(resultset.performance)

In [ ]:
!benchmark_app -m ./pot_model/openvino.xml  -i ../data/airport/val

## NNCF

### Compress PyTorch model with NNCF

In [ ]:
environment.model = torch_model
nncf_model = ModelEntity(dataset, environment.get_model_configuration())
nncf_task = get_impl_class(model_template.entrypoints.nncf)(environment)
nncf_task.optimize(
        OptimizationType.NNCF,
        dataset,
        nncf_model,
        OptimizationParameters(),
    )
save_model_data(nncf_model, "./nncf_torch_model")

### Evaluate the compressed model

In [ ]:
predicted_validation_dataset = nncf_task.infer(
    validation_dataset.with_empty_annotations(),
    InferenceParameters(is_evaluation=True))
resultset = ResultSetEntity(
    model=nncf_model,
    ground_truth_dataset=validation_dataset,
    prediction_dataset=predicted_validation_dataset,
)
nncf_task.evaluate(resultset)
print(resultset.performance)

### Export NNCF PyTorch to OpenVINO

In [ ]:
nncf_exported_model = ModelEntity(None, environment.get_model_configuration())
nncf_task.export(ExportType.OPENVINO, nncf_exported_model)
save_model_data(nncf_exported_model, "./nncf_exported_model")

### Evaluate NNCF OpenVINO model

In [ ]:
environment.model = nncf_exported_model
ov_task = get_impl_class(model_template.entrypoints.openvino)(environment)
predicted_validation_dataset = ov_task.infer(
    validation_dataset.with_empty_annotations(),
    InferenceParameters(is_evaluation=True))
resultset = ResultSetEntity(
    model=nncf_exported_model,
    ground_truth_dataset=validation_dataset,
    prediction_dataset=predicted_validation_dataset,
)
ov_task.evaluate(resultset)
print(resultset.performance)

## Draw bounding boxes

In [ ]:
import IPython
import PIL

for predictions, item in zip(predicted_validation_dataset, validation_dataset.with_empty_annotations()):
    image = item.numpy.astype(np.uint8)
    for box in predictions.annotation_scene.shapes:
        x1 = int(box.x1 * image.shape[1])
        x2 = int(box.x2 * image.shape[1])
        y1 = int(box.y1 * image.shape[0])
        y2 = int(box.y2 * image.shape[0])
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 3)  
    IPython.display.display(PIL.Image.fromarray(image))